In [1]:
!pip install split_folders

In [2]:
import os
import splitfolders
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models

loc = "/kaggle/input/autism-image-data/AutismDataset/consolidated"
os.makedirs('output', exist_ok=True)
os.makedirs('output/train', exist_ok=True)
os.makedirs('output/val', exist_ok=True)
os.makedirs('output/test', exist_ok=True)
splitfolders.ratio(loc, output="output", seed=42, ratio=(0.80, 0.1, 0.1))

train_dir = "output/train"
test_dir = "output/test"
val_dir = "output/val"

def convert_to_grayscale(image, label):
    image = tf.image.rgb_to_grayscale(image)
    return image, label

train_data = image_dataset_from_directory(
    train_dir, 
    batch_size=32, 
    image_size=(224, 224), 
    label_mode='categorical', 
    shuffle=True, 
    seed=42
)

test_data = image_dataset_from_directory(
    test_dir, 
    batch_size=32, 
    image_size=(224, 224), 
    label_mode='categorical', 
    shuffle=False, 
    seed=42
)

val_data = image_dataset_from_directory(
    val_dir, 
    batch_size=32, 
    image_size=(224, 224), 
    label_mode='categorical', 
    shuffle=False, 
    seed=42
)

def create_cnn_model(input_shape=(224, 224, 3), num_classes=2):
    model = models.Sequential()
    
    model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model
model=create_cnn_model()
model.summary()

history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc}")


Copying files: 2940 files [00:19, 153.91 files/s]


Found 2352 files belonging to 2 classes.
Found 294 files belonging to 2 classes.
Found 294 files belonging to 2 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

Epoch 1/10


I0000 00:00:1726290157.493772      80 service.cc:145] XLA service 0x7ddf5000b790 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726290157.493836      80 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1726290157.493841      80 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


 2/74 ━━━━━━━━━━━━━━━━━━━━ 7s 104ms/step - accuracy: 0.4922 - loss: 4.3998 

I0000 00:00:1726290166.110227      80 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74/74 ━━━━━━━━━━━━━━━━━━━━ 22s 143ms/step - accuracy: 0.6301 - loss: 7.3443 - val_accuracy: 0.5000 - val_loss: 4.7386
Epoch 2/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - accuracy: 0.6717 - loss: 0.7293 - val_accuracy: 0.7177 - val_loss: 0.9025
Epoch 3/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - accuracy: 0.7174 - loss: 0.6088 - val_accuracy: 0.7075 - val_loss: 1.1125
Epoch 4/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - accuracy: 0.7457 - loss: 0.4794 - val_accuracy: 0.7415 - val_loss: 0.5776
Epoch 5/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy: 0.7757 - loss: 0.4301 - val_accuracy: 0.7279 - val_loss: 0.7793
Epoch 6/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy: 0.7866 - loss: 0.4512 - val_accuracy: 0.7449 - val_loss: 0.6423
Epoch 7/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy: 0.8087 - loss: 0.4118 - val_accuracy: 0.7279 - val_loss: 0.8087
Epoch 8/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.8255 - loss: 0.3706 - val_accuracy: 0.7483 - val_loss: 

In [3]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

def create_resnet_model(input_shape=(224, 224, 3), num_classes=2):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

resnet_model = create_resnet_model()
resnet_model.summary()

history_resnet = resnet_model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

test_loss_resnet, test_acc_resnet = resnet_model.evaluate(test_data)
print(f"ResNet Test Accuracy: {test_acc_resnet}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ ?                      │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

Epoch 1/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 113s 704ms/step - accuracy: 0.6515 - loss: 0.8315 - val_accuracy: 0.5204 - val_loss: 52.2262
Epoch 2/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 22s 294ms/step - accuracy: 0.7438 - loss: 0.5530 - val_accuracy: 0.6429 - val_loss: 1.1506
Epoch 3/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 22s 304ms/step - accuracy: 0.7852 - loss: 0.4556 - val_accuracy: 0.6020 - val_loss: 1.9276
Epoch 4/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 22s 297ms/step - accuracy: 0.8237 - loss: 0.4033 - val_accuracy: 0.5510 - val_loss: 10.8759
Epoch 5/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 22s 293ms/step - accuracy: 0.8370 - loss: 0.4071 - val_accuracy: 0.6599 - val_loss: 3.9554
Epoch 6/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 22s 296ms/step - accuracy: 0.8675 - loss: 0.3302 - val_accuracy: 0.6463 - val_loss: 1.1351
Epoch 7/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 22s 298ms/step - accuracy: 0.9013 - loss: 0.2680 - val_accuracy: 0.6701 - val_loss: 0.8636
Epoch 8/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 22s 297ms/step - accuracy: 0.9100 - loss: 0.2360 - val_a

In [4]:
from tensorflow.keras.applications import MobileNet

def create_mobilenet_model(input_shape=(224, 224, 3), num_classes=2):
    base_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
    
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(128, activation='relu', kernel_initializer='he_normal'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

mobilenet_model = create_mobilenet_model()
mobilenet_model.summary()

history_mobilenet = mobilenet_model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

test_loss_mobilenet, test_acc_mobilenet = mobilenet_model.evaluate(test_data)
print(f"MobileNet Test Accuracy: {test_acc_mobilenet}")


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_224 (Functional) │ ?                      │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,228,864 (12.32 MB)

 Trainable params: 3,206,976 (12.23 MB)

 Non-trainable params: 21,888 (85.50 KB)

Epoch 1/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 53s 319ms/step - accuracy: 0.6948 - loss: 0.7460 - val_accuracy: 0.5000 - val_loss: 4.7157
Epoch 2/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 7s 94ms/step - accuracy: 0.8524 - loss: 0.3528 - val_accuracy: 0.8197 - val_loss: 0.4946
Epoch 3/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 7s 95ms/step - accuracy: 0.8987 - loss: 0.2591 - val_accuracy: 0.8163 - val_loss: 0.5145
Epoch 4/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 7s 95ms/step - accuracy: 0.9117 - loss: 0.2218 - val_accuracy: 0.7211 - val_loss: 0.8934
Epoch 5/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 7s 94ms/step - accuracy: 0.9433 - loss: 0.1536 - val_accuracy: 0.7823 - val_loss: 0.7765
Epoch 6/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 7s 93ms/step - accuracy: 0.9548 - loss: 0.1200 - val_accuracy: 0.7415 - val_loss: 1.7453
Epoch 7/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 7s 92ms/step - accuracy: 0.9474 - loss: 0.1628 - val_accuracy: 0.5748 - val_loss: 3.0412
Epoch 8/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 7s 91ms/step - accuracy: 0.9727 - loss: 0.0829 - val_accuracy: 0.8401 -

In [5]:
from tensorflow.keras.applications import EfficientNetB0

def create_efficientnet_model(input_shape=(224, 224, 3), num_classes=2):
    # Load EfficientNetB0 without the top layers
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

# Create and summarize the model
model = create_efficientnet_model()
model.summary()

# Training the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc}")


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ ?                      │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,049,571 (15.45 MB)

 Trainable params: 4,007,548 (15.29 MB)

 Non-trainable params: 42,023 (164.16 KB)

Epoch 1/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 154s 933ms/step - accuracy: 0.7059 - loss: 0.5702 - val_accuracy: 0.7857 - val_loss: 0.5275
Epoch 2/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 10s 137ms/step - accuracy: 0.8578 - loss: 0.3440 - val_accuracy: 0.7143 - val_loss: 0.9063
Epoch 3/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 10s 140ms/step - accuracy: 0.9107 - loss: 0.2208 - val_accuracy: 0.5816 - val_loss: 2.2632
Epoch 4/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 10s 139ms/step - accuracy: 0.9261 - loss: 0.1808 - val_accuracy: 0.7823 - val_loss: 0.7947
Epoch 5/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 10s 138ms/step - accuracy: 0.9469 - loss: 0.1407 - val_accuracy: 0.8265 - val_loss: 0.5361
Epoch 6/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 10s 136ms/step - accuracy: 0.9528 - loss: 0.1158 - val_accuracy: 0.8231 - val_loss: 0.7541
Epoch 7/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 10s 134ms/step - accuracy: 0.9554 - loss: 0.1032 - val_accuracy: 0.8367 - val_loss: 0.5447
Epoch 8/10
74/74 ━━━━━━━━━━━━━━━━━━━━ 10s 136ms/step - accuracy: 0.9660 - loss: 0.1060 - val_acc